# transformerのインストール


In [ ]:
!pip install keras-transformer

  Created wheel for keras-transformer: filename=keras_transformer-0.37.0-cp36-none-any.whl size=12942 sha256=92059b0e49cfa083a7c3b023b54eec2013461faf0770994ff8642c2cac3010cf
  Stored in directory: /root/.cache/pip/wheels/2a/f9/31/2a3289e948852ce0dd3fcd94c34bbc7eb9628842cb7110a87b
  Created wheel for keras-pos-embd: filename=keras_pos_embd-0.11.0-cp36-none-any.whl size=7554 sha256=57770b041be5278da8bd673d33554aaff198ebb6a08ade67f0497453217a07cd
  Stored in directory: /root/.cache/pip/wheels/5b/a1/a0/ce6b1d49ba1a9a76f592e70cf297b05c96bc9f418146761032
  Created wheel for keras-multi-head: filename=keras_multi_head-0.27.0-cp36-none-any.whl size=15611 sha256=4071b13901e9e977eeb7134de5a878172bd113303c94a271c2cb91de455c8588
  Stored in directory: /root/.cache/pip/wheels/b5/b4/49/0a0c27dcb93c13af02fea254ff51d1a43a924dd4e5b7a7164d
  Created wheel for keras-layer-normalization: filename=keras_layer_normalization-0.14.0-cp36-none-any.whl size=5268 sha256=43c00052a220c5e8f29c3680c1defc3ebc81f76d53

# tokenizerのインストール

In [ ]:
!pip install sentencepiece

     |████████████████████████████████| 1.1MB 2.8MB/s 


# ドライブのマウント

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


#コーパスのダウンロードと整形

In [ ]:
!git clone https://github.com/knok/make-meidai-dialogue.git

Cloning into 'make-meidai-dialogue'...
remote: Enumerating objects: 35, done.
remote: Total 35 (delta 0), reused 0 (delta 0), pack-reused 35
Unpacking objects: 100% (35/35), done.


In [ ]:
cd "/content/drive/My Drive/Colab Notebooks/make-meidai-dialogue"

/content/drive/My Drive/Colab Notebooks/make-meidai-dialogue


In [ ]:
!make all

unzip -x nucc.zip
Archive:  nucc.zip
replace nucc/data001.txt? [y]es, [n]o, [A]ll, [N]one, [r]ename: A
  inflating: nucc/data001.txt        
  inflating: nucc/data002.txt        
  inflating: nucc/data003.txt        
  inflating: nucc/data004.txt        
  inflating: nucc/data005.txt        
  inflating: nucc/data006.txt        
  inflating: nucc/data007.txt        
  inflating: nucc/data008.txt        
  inflating: nucc/data009.txt        
  inflating: nucc/data010.txt        
  inflating: nucc/data011.txt        
  inflating: nucc/data012.txt        
  inflating: nucc/data013.txt        
  inflating: nucc/data014.txt        
  inflating: nucc/data015.txt        
  inflating: nucc/data016.txt        
  inflating: nucc/data017.txt        
  inflating: nucc/data018.txt        
  inflating: nucc/data019.txt        
  inflating: nucc/data020.txt        
  inflating: nucc/data021.txt        
  inflating: nucc/data022.txt        
  inflating: nucc/data023.txt        
  inflating: nucc/data0

In [ ]:
cd "/content/drive/My Drive/Colab Notebooks"

/content/drive/My Drive/Colab Notebooks


#コーパスをinputとoutputに分割する

In [ ]:
input_corpus = []
output_corpus = []
for_spm_corpus = []
with open('/content/drive/My Drive/Colab Notebooks/make-meidai-dialogue/sequence.txt') as f:
  for s_line in f:

    if s_line.startswith('input: '):
      input_corpus.append(s_line[6:])
      for_spm_corpus.append(s_line[6:])

    elif s_line.startswith('output: '):
      output_corpus.append(s_line[7:])
      for_spm_corpus.append(s_line[7:])

with open('/content/drive/My Drive/Colab Notebooks/input_corpus.txt', 'w') as f:
  f.writelines(input_corpus)

with open('/content/drive/My Drive/Colab Notebooks/output_corpus.txt', 'w') as f:
  f.writelines(output_corpus)

with open('/content/drive/My Drive/Colab Notebooks/spm_corpus.txt', 'w') as f:
  f.writelines(for_spm_corpus)

# 分割したコーパスを単語分割する

In [ ]:
import sentencepiece as spm

# train sentence piece
spm.SentencePieceTrainer.Train("--input=spm_corpus.txt --model_prefix=trained_model --vocab_size=8000 --bos_id=1 --eos_id=2 --pad_id=0 --unk_id=5")

In [ ]:
sp = spm.SentencePieceProcessor()
sp.Load("trained_model.model")

#test
print(sp.EncodeAsPieces("ああそういうことね"))
print(sp.EncodeAsPieces("なるほどわかった"))
print(sp.EncodeAsPieces("つまり、そのあなたの言いたいことはつまりそういうことですか？"))
print(sp.DecodeIds([0,1,2,3,4,5]))

['▁ああ', 'そういうこと', 'ね']
['▁なるほど', 'わかった']
['▁', 'つまり', '、', 'その', 'あなた', 'の', '言い', 'たい', 'ことは', 'つまり', 'そういうこと', 'ですか', '?']
、。 ⁇ 


# パディングとtransformerに適した形式に変更する


In [ ]:
import numpy as np

# Generate toy data
encoder_inputs_no_padding = []
encoder_inputs, decoder_inputs, decoder_outputs = [], [], []
max_token_size = 168

with open('/content/drive/My Drive/Colab Notebooks/input_corpus.txt') as input_tokens, open('/content/drive/My Drive/Colab Notebooks/output_corpus.txt') as output_tokens:
  #コーパスから一行ずつ読み込む
  input_tokens = input_tokens.readlines()
  output_tokens = output_tokens.readlines()

  for input_token, output_token in zip(input_tokens, output_tokens):
    if input_token or output_token:
      encode_tokens, decode_tokens = sp.EncodeAsPieces(input_token), sp.EncodeAsPieces(output_token)
      #パディングする
      encode_tokens = ['<s>'] + encode_tokens + ['</s>'] + ['<pad>'] * (max_token_size - len(encode_tokens))
      output_tokens = decode_tokens + ['</s>', '<pad>'] + ['<pad>'] * (max_token_size - len(decode_tokens))
      decode_tokens = ['<s>'] + decode_tokens + ['</s>']  + ['<pad>'] * (max_token_size - len(decode_tokens))

      
      encode_tokens = list(map(lambda x: sp.piece_to_id(x), encode_tokens))
      decode_tokens = list(map(lambda x: sp.piece_to_id(x), decode_tokens))
      output_tokens = list(map(lambda x: [sp.piece_to_id(x)], output_tokens))

      encoder_inputs_no_padding.append(input_token)
      encoder_inputs.append(encode_tokens)
      decoder_inputs.append(decode_tokens)
      decoder_outputs.append(output_tokens)
    else:
      break

#学習モデルへの入力用に変換する
X = [np.asarray(encoder_inputs), np.asarray(decoder_inputs)]
Y = np.asarray(decoder_outputs)

# 学習する

In [ ]:
from keras_transformer import get_model

# Build the model
model = get_model(
    token_num=sp.GetPieceSize(),
    embed_dim=256,
    encoder_num=3,
    decoder_num=2,
    head_num=2,
    hidden_dim=3072,
    attention_activation='relu',
    feed_forward_activation='relu',
    dropout_rate=0.05,
    embed_weights=np.random.random((sp.GetPieceSize(), 256)),
)
model.compile(
    optimizer='adam',
    loss='sparse_categorical_crossentropy',
)
model.summary()

# Train the model
model.fit(
    x=X,
    y=Y,
    epochs=5,
)

Model: "model_2"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
Decoder-Input (InputLayer)      (None, None)         0                                            
__________________________________________________________________________________________________
Encoder-Input (InputLayer)      (None, None)         0                                            
__________________________________________________________________________________________________
Token-Embedding (EmbeddingRet)  [(None, None, 256),  2048000     Encoder-Input[0][0]              
                                                                 Decoder-Input[0][0]              
__________________________________________________________________________________________________
Encoder-Embedding (TrigPosEmbed (None, None, 256)    0           Token-Embedding[0][0]      

# 推論

In [ ]:
from keras_transformer import decode

input = "明日の天気は？"
encode = sp.EncodeAsIds(input)

decoded = decode(
    model,
    encode,
    start_token=sp.bos_id(),
    end_token=sp.eos_id(),
    pad_token=sp.pad_id(),
    max_len=170
)



In [ ]:
decoded = np.array(decoded,dtype=int)
decoded = decoded.tolist()
print(sp.decode(decoded))